# AgentConfig for Structured Outputs

AG2 now supports an official `AgentConfig` class (from `autogen.llm_config`) for advanced per-agent configuration, such as specifying structured output schemas or agent-level API settings. This notebook demonstrates how to use `AgentConfig` to enable schema-based structured outputs in your agents.

````{=mdx}
:::info Requirements
Install `ag2`:

pip install -U ag2[openai]
```

> **Note:** If you have been using `autogen` or `ag2`, all you need to do is upgrade it using:  
>
> pip install -U autogen
> ```
> or  
>
> pip install -U ag2
> ```
> as `autogen`, and `ag2` are aliases for the same PyPI package.  


For more information, please refer to the [installation guide](https://docs.ag2.ai/latest/docs/user-guide/basic-concepts/installing-ag2).
:::
````

## What is AgentConfig?

`AgentConfig` is a dedicated configuration class designed for per-agent settings in AG2. It provides a clean and explicit way to configure agent-specific features, particularly structured outputs through the `response_format` parameter.

### Key Features:

- **Per-Agent Configuration**: Configure settings specific to individual agents without affecting others
- **Structured Output Support**: Specify Pydantic models as `response_format` to enforce structured responses
- **API Type Control**: Set `api_type` directly at the agent level
- **Extensible Design**: Accepts additional configuration parameters through keyword arguments

### Advantages over LLMConfig for Agent-Level Configuration:

1. **Simplicity**: `AgentConfig` is designed specifically for agent-level configuration, making it more intuitive for per-agent settings
2. **Clarity**: Explicitly indicates that the configuration is agent-specific, improving code readability
3. **Flexibility**: Allows different agents in the same workflow to have different API types and response formats
4. **Preference for Structured Outputs**: `AgentConfig` is often preferred for setting up structured outputs, as it provides a clean interface for schema-based results

You can pass either `llm_config` (using `LLMConfig`) or `agent_config` (using `AgentConfig`) to agents, depending on your needs. Use `AgentConfig` when you want per-agent structured output or need to control how a particular agent interacts with the LLM API.

## Example 1: Two-Agent Chat with Structured Outputs

This example demonstrates how to use `AgentConfig` with structured outputs in a simple two-agent conversation. We'll create an assistant agent that responds with structured joke data, and a user proxy agent to interact with it.

In [ ]:
import os

from pydantic import BaseModel

from autogen import AgentConfig, AssistantAgent, LLMConfig, UserProxyAgent

llm_config = LLMConfig(
    config_list=[
        {
            "model": "gpt-4o-mini",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "api_type": "openai",
        }
    ]
)


# Define a structured output schema for the assistant's responses
class JokeOutputSchema(BaseModel):
    joke: str
    explanation: str


# Create AgentConfig with structured output and API type
assistant_config = AgentConfig(
    response_format=JokeOutputSchema,
    api_type="openai",
)

# Create the assistant agent with AgentConfig
assistant = AssistantAgent(
    "assistant",
    agent_config=assistant_config,
    system_message="You are a helpful assistant that tells jokes and explains them. Always structure your responses according to the JokeOutputSchema.",
    llm_config=llm_config,
)

# Create the user proxy agent (no structured output needed)
user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config=False,
    human_input_mode="NEVER",
)

### Start the Two-Agent Chat

Now let's initiate a conversation between the user proxy and the assistant. The assistant will respond with structured data matching our `JokeOutputSchema`.

In [ ]:
# Start the chat

result = user_proxy.run(
    assistant,
    message="Tell me a joke about NVDA and TESLA stock prices.",
    max_turns=1,
)
result.process()

## Example 2: GroupChat with Multiple Agents Using Structured Outputs

In this example, we'll create a GroupChat with multiple agents, each potentially using `AgentConfig` for structured outputs. This demonstrates how different agents can have different configurations in a multi-agent conversation.

We'll create a scenario where:
- A **Reviewer** agent provides structured feedback
- A **Summarizer** agent provides structured summaries
- A **User Proxy** agent coordinates the conversation

In [ ]:
import os

from pydantic import BaseModel

from autogen import AgentConfig, AssistantAgent, LLMConfig, UserProxyAgent
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns.auto import AutoPattern


class Answer(BaseModel):
    answer: str


llm_config = LLMConfig(
    config_list=[
        {
            "model": "gpt-4o-mini",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "api_type": "openai",
            "response_format": Answer,
        }
    ]
)


# Define structured output schema for the Reviewer agent
class ReviewFeedback(BaseModel):
    rating: int  # 1-5 scale
    strengths: list[str]
    weaknesses: list[str]
    recommendation: str


# Define structured output schema for the Summarizer agent
class SummaryOutput(BaseModel):
    title: str
    key_points: list[str]
    conclusion: str


# Create AgentConfig for the Reviewer agent
reviewer_config = AgentConfig(
    response_format=ReviewFeedback,
)

# Create AgentConfig for the Summarizer agent
summarizer_config = AgentConfig(
    response_format=SummaryOutput,
)

# Create the Reviewer agent with structured output
reviewer = AssistantAgent(
    name="reviewer",
    agent_config=reviewer_config,
    system_message="You are a thorough reviewer. Analyze content and provide structured feedback with ratings, strengths, weaknesses, and recommendations.",
    llm_config=llm_config,
)

# Create the Summarizer agent with structured output
summarizer = AssistantAgent(
    name="summarizer",
    agent_config=summarizer_config,
    system_message="You are a concise summarizer. Create structured summaries with titles, key points, and conclusions.",
    llm_config=llm_config,
)

# Create the User Proxy agent (no structured output needed)
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    human_input_mode="NEVER",
    llm_config=llm_config,
)

# Create GroupChat with all agents
pattern = AutoPattern(
    initial_agent=user_proxy,
    agents=[reviewer, summarizer, user_proxy],
    group_manager_args={
        "llm_config": llm_config,  # Use same config for group manager
    },
)

print("AutoPattern created with structured output agent!")

### Start the GroupChat

Now let's initiate the GroupChat. The reviewer and summarizer agents will provide structured responses according to their respective schemas.

In [ ]:
# Start the GroupChat conversation
result, context, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="Please review and summarize this idea: Creating an AI-powered assistant for code review that provides structured feedback on code quality, security, and best practices.",
    max_rounds=5,
)

## Key Takeaways

1. **AgentConfig is Ideal for Structured Outputs**: When you need to enforce structured responses from agents, `AgentConfig` provides a clean and explicit way to specify the `response_format` parameter.

2. **Per-Agent Flexibility**: Different agents in the same workflow can have different structured output schemas, allowing for specialized agent roles with specific response formats.

3. **Simplicity**: `AgentConfig` simplifies agent-level configuration, making it easier to understand and maintain code that uses structured outputs.

4. **Integration with GroupChat**: `AgentConfig` works seamlessly with GroupChat, allowing multiple agents with different structured output schemas to collaborate effectively.

5. **Clean System Messages**: With structured outputs, you don't need to include formatting instructions in system messages - the schema enforces the structure automatically.

Remember: You can use either `llm_config` (with `LLMConfig`) or `agent_config` (with `AgentConfig`) depending on your needs. For per-agent structured outputs and clearer agent-specific configuration, `AgentConfig` is the preferred choice.